In [19]:
import pandas as pd
import json
import re
import string
import pymorphy2
from scipy.sparse import *
import stop_words
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [20]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [6]:
EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                                '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—', '‘', 
                                '\x7f', '\xad', '¤', '\xa0'])
GRAMMS = ['NOUN', 'ADJF', 'ADJS', 'PRTF', 'PRTS', 'GRND', 'ADVB']
DATA_PATH = 'life_news_20k.json'

In [7]:
news = []
with open(DATA_PATH) as file:
    for line in file:
        news.append(json.loads(line))
data = pd.DataFrame(news)
data

,link,title,category,tags,text
0,https://life.ru/p/1471948,"Стало известно, кто захоронен в безымянных сар...",tehnologii,"[Новости, захоронения, кремль, Интересное, Нау...","Учёные провели исследования и выяснили, кто за..."
1,https://life.ru/p/1471901,"Google пообещал ""воскресить"" старые компьютеры...",tehnologii,"[Новости, google, Сервисы и приложения, Наука ...",Google анонсировал новую версию фирменной Chro...
2,https://life.ru/p/1471856,Минцифры опровергло сообщения о планах ввести ...,tehnologii,"[Новости, Минцифры, Экономика, Наука и Техноло...","Министерство цифрового развития, связи и массо..."
3,https://life.ru/p/1471819,TikTok приступил к созданию представительства ...,tehnologii,"[Новости, tiktok, Социальные сети и мессенджер...",TikTok приступил к созданию представительства ...
4,https://life.ru/p/1471813,В России могут ввести налог для Steam и PlaySt...,tehnologii,"[Новости, playstationstore, steam, Наука и Тех...",В России в целях поддержки отечественных видео...
...,...,...,...,...,...
20451,https://life.ru/p/1451297,Косачёв счёл угрозу санкций США против России ...,politika,"[Новости, США, Санкции, Украина, Мировая полит...",Вице-спикер Совета Федерации по международным ...
20452,https://life.ru/p/1451296,Баку подтвердил предстоящую встречу Алиева и П...,politika,"[Новости, Азербайджан, Армения, Никол Пашинян,...",Азербайджанская сторона дала согласие на предл...
20453,https://life.ru/p/1451295,"WSJ: Путин разрушил миф об ""угасающей"" России ...",politika,"[Новости, Путин, США, Мировая политика, Политика]",Западным странам пора отказаться от самолюбивы...
20454,https://life.ru/p/1451286,Глава Пентагона назвал Иран главной угрозой дл...,politika,"[Новости, США, Иран, Мировая политика, Политика]",Глава Пентагона Ллойд Остин назвал Иран одной ...


In [10]:
corpus, texts = [], []

for message in log_progress(data.text.dropna().values):
    sp = message.split('\n')
    corpus += sp
    texts += sp
print(len(corpus), len(texts))

20456 20456


In [11]:
print(corpus[2345])
print(len(corpus[2345]))

Экран — один из плюсов линейки Galaxy A и одно из достоинств модели A52. Картинка на экране Super AMOLED с разрешением 2400x1080 будет динамичной и яркой, а датчик отпечатка пальца почти не занимает места на корпусе. У лидера бюджетных камерофонов от "Самсунга" четыре основные камеры с разрешением 64+8+5+5 Мп. Они позволяют делать качественные снимки на уровне более дорогих конкурентов. Аккумулятор в Galaxy A52 тоже неплох — 4500 мА*ч.   Процессор от компании Qualcomm, по отзывам пользователей, позволяет запускать видео в хорошем качестве как в "Ютубе", так и на других площадках. При интенсивном использовании смартфон нагревается слабее своих конкурентов, а хороший стереозвук на внешних динамиках позволяет в большинстве случаев обойтись без наушников. Стоимость устройства — 27–29 тысяч рублей.   Переиздание модели 2019 года получилось удачным. Процессор — MediaTek Helio G85 с хорошей производительностью — позволяет запускать даже требовательные мобильные игры. С повседневными задачами 

In [12]:
corpus = [re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u'', doc) for doc in corpus]
corpus = [re.sub(' +' , ' ', doc) for doc in corpus]
corpus = np.asarray([doc.strip().strip('\t').replace('\n', u'') for doc in corpus])

In [13]:
regex_punct = re.compile('[%s]' % re.escape(string.punctuation))
regex_dig = re.compile('[%s]' % re.escape(string.digits))
regex_symb = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))
regex_struct = re.compile('[%s]' % string.printable + string.whitespace)
emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"
                       u"\U0001F300-\U0001F5FF"
                       u"\U0001F680-\U0001F6FF"
                       u"\U0001F1E0-\U0001F1FF"
                       u"\U00002500-\U00002BEF"
                       u"\U00002702-\U000027B0"
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       u"\U0001f926-\U0001f937"
                       u"\U00010000-\U0010ffff"
                       u"\u2640-\u2642"
                       u"\u2600-\u2B55"
                       u"\u200d"
                       u"\u23cf"
                       u"\u23e9"
                       u"\u231a"
                       u"\ufe0f"
                       u"\u3030"
                       "]+", flags=re.UNICODE)

In [14]:
corpus = [regex_punct.sub('', doc) for doc in corpus]
corpus = [regex_dig.sub('', doc) for doc in corpus]
corpus = [regex_symb.sub(' ', doc) for doc in corpus]
corpus = [regex_struct.sub('', doc) for doc in corpus]
corpus = [emoji_pattern.sub('', doc) for doc in corpus]
corpus = [re.sub(' +' , ' ', doc.strip()) for doc in corpus]
print(len(corpus))

20456


In [15]:
corpus = [re.sub('<.*?>+', '', doc) for doc in corpus]
corpus = [re.sub(r'http\s+|www.\s+', '', doc) for doc in corpus]
corpus = [re.sub(r'www.\S+', '', doc) for doc in corpus]
corpus = [re.sub(r'http\S+', '', doc) for doc in corpus]

In [16]:
 morph = pymorphy2.MorphAnalyzer()

In [17]:
def lemmatize_text(text):
    new_text = str()
    for word in text.split(' '):
        new_word = morph.parse(word)[0].normal_form
        new_text += new_word + ' '
    new_text = new_text[:-1]
    return new_text

In [18]:
corpus = list(map(lemmatize_text, corpus))

In [22]:
stop = stopwords.words('russian')
stop[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [23]:
corpus = list(map(lambda x: ' '.join([word for word in x.split() if word not in (stop)]), corpus))

In [24]:
corpus[2345]

'экран плюс линейка galaxy a достоинство модель a картинка экран super amoled разрешение x динамичный яркий датчик отпечаток палец занимать место корпус лидер бюджетный камерофон самсунг четыре основной камера разрешение мп позволять делать качественный снимок уровень дорогой конкурент аккумулятор galaxy a неплохой мач процессор компания qualcomm отзыв пользователь позволять запускать видео хороший качество ютуба площадка интенсивный использование смартфон нагреваться слабый свой конкурент хороший стереозвук внешний динамик позволять большинство случай обойтись наушник стоимость устройство – тысяча рубль переиздание модель год получиться удачный процессор mediatek helio g хороший производительность позволять запускать требовательный мобильный игра повседневный задача проблема приложение открываться загружаться молниеносно экран стандартный ценовый уровень тип матрица ips диагональ дюйм разрешение full hd ёмкость аккумулятор мач поддерживаться быстрый зарядка мощность вт заряд хватать п

In [28]:
news = []
with open(DATA_PATH) as file:
    for line in file:
        news.append(json.loads(line))
init_data = pd.DataFrame(news)
init_data

,link,title,category,tags,text
0,https://life.ru/p/1471948,"Стало известно, кто захоронен в безымянных сар...",tehnologii,"[Новости, захоронения, кремль, Интересное, Нау...","Учёные провели исследования и выяснили, кто за..."
1,https://life.ru/p/1471901,"Google пообещал ""воскресить"" старые компьютеры...",tehnologii,"[Новости, google, Сервисы и приложения, Наука ...",Google анонсировал новую версию фирменной Chro...
2,https://life.ru/p/1471856,Минцифры опровергло сообщения о планах ввести ...,tehnologii,"[Новости, Минцифры, Экономика, Наука и Техноло...","Министерство цифрового развития, связи и массо..."
3,https://life.ru/p/1471819,TikTok приступил к созданию представительства ...,tehnologii,"[Новости, tiktok, Социальные сети и мессенджер...",TikTok приступил к созданию представительства ...
4,https://life.ru/p/1471813,В России могут ввести налог для Steam и PlaySt...,tehnologii,"[Новости, playstationstore, steam, Наука и Тех...",В России в целях поддержки отечественных видео...
...,...,...,...,...,...
20451,https://life.ru/p/1451297,Косачёв счёл угрозу санкций США против России ...,politika,"[Новости, США, Санкции, Украина, Мировая полит...",Вице-спикер Совета Федерации по международным ...
20452,https://life.ru/p/1451296,Баку подтвердил предстоящую встречу Алиева и П...,politika,"[Новости, Азербайджан, Армения, Никол Пашинян,...",Азербайджанская сторона дала согласие на предл...
20453,https://life.ru/p/1451295,"WSJ: Путин разрушил миф об ""угасающей"" России ...",politika,"[Новости, Путин, США, Мировая политика, Политика]",Западным странам пора отказаться от самолюбивы...
20454,https://life.ru/p/1451286,Глава Пентагона назвал Иран главной угрозой дл...,politika,"[Новости, США, Иран, Мировая политика, Политика]",Глава Пентагона Ллойд Остин назвал Иран одной ...


In [29]:
init_data['text'] = corpus
js = json.loads(init_data.to_json(force_ascii=False, orient='records'))
init_data

,link,title,category,tags,text
0,https://life.ru/p/1471948,"Стало известно, кто захоронен в безымянных сар...",tehnologii,"[Новости, захоронения, кремль, Интересное, Нау...",учёный провести исследование выяснить захорони...
1,https://life.ru/p/1471901,"Google пообещал ""воскресить"" старые компьютеры...",tehnologii,"[Новости, google, Сервисы и приложения, Наука ...",google анонсировать новый версия фирменный chr...
2,https://life.ru/p/1471856,Минцифры опровергло сообщения о планах ввести ...,tehnologii,"[Новости, Минцифры, Экономика, Наука и Техноло...",министерство цифровой развитие связь массовый ...
3,https://life.ru/p/1471819,TikTok приступил к созданию представительства ...,tehnologii,"[Новости, tiktok, Социальные сети и мессенджер...",tiktok приступить создание представительство р...
4,https://life.ru/p/1471813,В России могут ввести налог для Steam и PlaySt...,tehnologii,"[Новости, playstationstore, steam, Наука и Тех...",россия цель поддержка отечественный видеоигра ...
...,...,...,...,...,...
20451,https://life.ru/p/1451297,Косачёв счёл угрозу санкций США против России ...,politika,"[Новости, США, Санкции, Украина, Мировая полит...",вицеспикер совет федерация международный дело ...
20452,https://life.ru/p/1451296,Баку подтвердил предстоящую встречу Алиева и П...,politika,"[Новости, Азербайджан, Армения, Никол Пашинян,...",азербайджанский сторона дать согласие предложе...
20453,https://life.ru/p/1451295,"WSJ: Путин разрушил миф об ""угасающей"" России ...",politika,"[Новости, Путин, США, Мировая политика, Политика]",западный страна пора отказаться самолюбивый уб...
20454,https://life.ru/p/1451286,Глава Пентагона назвал Иран главной угрозой дл...,politika,"[Новости, США, Иран, Мировая политика, Политика]",глава пентагон ллойда остин назвать иран главн...


In [30]:
with open('life_news_preproc.json', 'w') as file:
    for i in js:
        file.write(json.dumps(i) + '\n')